In [1]:
# ---- load globals ----
source("R/globals.R", local = TRUE)

source("R/data_loading.R", local = TRUE)


source("R/module_rotate.R", local = TRUE)

library(plotly)       # ← 新增
library(base64enc)    # ← 新增（如果保留 data URI 路线）
source("R/module_select.R") 

source("R/download.R", local = TRUE)


source("R/plot.R", local = TRUE)

source("R/distance.R", local = TRUE)



Attaching package: ‘jsonlite’


The following object is masked from ‘package:shiny’:

    validate



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




In [2]:
# # ---- Shiny UI ----

ui <- fluidPage(
  titlePanel("Visium HE overlay + gene expression (Lv0)"),
  sidebarLayout(
    sidebarPanel(
      # 上传控件
      fileInput("matrix_file",   "上传 matrix.mtx(.gz)"),
      fileInput("barcodes_file", "上传 barcodes.tsv(.gz)"),
      fileInput("features_file", "上传 features.tsv(.gz)"),
      fileInput("positions_file","上传 tissue_positions_list.csv"),
      fileInput("scalef_file",   "上传 scalefactors_json.json"),
      fileInput("image_file",    "上传 HE 图像 (png/jpg)"),

      # 基因选择：初始为空，加载后用 server 端 updateSelectInput() 填
      selectInput("gene", "选择基因：", choices = character(0), selected = NULL),


      checkboxInput("log1p", "log1p 变换", value = TRUE),
      sliderInput("ptsize", "点大小", min = 1, max = 6, value = 3, step = 0.5),
      sliderInput("alpha", "点透明度", min = 0.1, max = 1, value = 0.9, step = 0.1),
      helpText("请先上传 10x 矩阵三件套（matrix/barcodes/features）再选择基因。"),

      tags$hr(),
      rotateImageUI("rot")
    ),


    mainPanel(
      # 选择工具（套索/矩形）
      radioButtons(
        "sel_mode", "选择工具",
        choices = c("套索 (lasso)" = "lasso", "矩形 (box)" = "select"),
        inline = TRUE, selected = "lasso"
      ),

      # 交互主图：HE 叠底 + spots（可圈选）
      plotly::plotlyOutput("he_plotly", height = "750px", width = "100%"),

      # 操作区：清空/下载/计数
      fluidRow(
        column(4, actionButton("clear_sel", "清空选择")),
        column(4, downloadButton("dl_sel", "下载所选 CSV")),
        column(4, downloadButton("dl_distance", "下载带符号距离")),
        column(4, verbatimTextOutput("sel_count"))
      ),

      hr(),
      verbatimTextOutput("dbg")
    )
    )
  )


In [5]:
server <- function(input, output, session) {
  rv <- reactiveValues(
    counts=NULL, gene_names=NULL, barcodes=NULL,
    pos=NULL, hires_scale=NULL, img=NULL, img_w=NULL, img_h=NULL, img_path=NULL
  )

  # ---- 加载 counts/features/barcodes ----
  observeEvent(list(input$matrix_file, input$barcodes_file, input$features_file), {
    req(input$matrix_file, input$barcodes_file, input$features_file)
    paths <- list(input$matrix_file$datapath, input$barcodes_file$datapath, input$features_file$datapath)
    message("Loading counts ...")
    res <- load_counts(paths[[1]], paths[[2]], paths[[3]])
    rv$counts     <- res$counts
    rv$gene_names <- res$gene_names
    rv$barcodes   <- res$barcodes
    updateSelectInput(session, "gene", choices = head(rv$gene_names, MAX_GENES),
                      selected = head(rv$gene_names, 1))
  })

  # ---- 加载 positions ----
  observeEvent(input$positions_file, {
    req(input$positions_file)
    message("Loading positions ...")
    pos <- load_positions(input$positions_file$datapath)
    if (!is.null(rv$counts)) {
      pos <- subset(pos, in_tissue == 1 & barcode %in% colnames(rv$counts))
    } else {
      pos <- subset(pos, in_tissue == 1)
    }
    rv$pos <- pos
  })

  # ---- 加载 scalefactor ----
  observeEvent(input$scalef_file, {
    req(input$scalef_file)
    message("Loading scalefactor ...")
    rv$hires_scale <- load_scalef(input$scalef_file$datapath)
    if (!is.null(rv$pos)) rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
  })

  # ---- 加载图像 ----
  observeEvent(input$image_file, {
    req(input$image_file)
    message("Loading image ...")
    im <- load_image(input$image_file$datapath)
    rv$img   <- im$img      # raster
    rv$img_w <- im$img_w
    rv$img_h <- im$img_h
    rv$img_path <- input$image_file$datapath
  })

  # ---- pos 更新后补映射 ----
  observeEvent(rv$pos, {
    if (!is.null(rv$pos) && !is.null(rv$hires_scale) && is.null(rv$pos$x_hires)) {
      rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
    }
  })

  expr_vec <- make_expr_vec(
    gene_input = reactive(input$gene),
    counts_rv  = reactive(rv$counts),
    pos_rv     = reactive(rv$pos),
    do_log1p   = reactive(isTRUE(input$log1p))
  )


  rot <- rotateImageServer("rot", image_path_reactive = reactive(rv$img_path))


  sel <- selectSpotsServer(
      "sel",
      pos_reactive  = reactive(rv$pos),      # 需要包含 barcode/x_hires/y_hires
      expr_reactive = expr_vec,              # 你已有的表达向量 reactive
      img_reactive  = current_img_obj        # 背景图（旋转后/原图皆可）
    )


  # 当前背景（优先旋转后）
  current_img_obj <- reactive({
    ro <- tryCatch(rot$get(), error = function(e) NULL)
    if (!is.null(ro)) ro else if (!is.null(rv$img)) list(img=rv$img, img_w=rv$img_w, img_h=rv$img_h) else NULL
  })

  # 主图：HE 底图 + spot；在同图内圈选
  output$he_plotly <- plotly::renderPlotly({
    req(rv$pos, current_img_obj())
    build_he_plotly_figure(
      pos        = rv$pos,
      expr       = expr_vec(),
      img_obj    = current_img_obj(),
      ptsize     = input$ptsize,
      alpha      = input$alpha,
      gene_label = input$gene
    ) %>%
    plotly::layout(dragmode = if (isTruthy(input$sel_mode) && input$sel_mode=="select") "select" else "lasso")
  })



  selected_idx <- reactiveVal(integer(0))
  attach_plotly_selection_handlers(
    output_id       = "he_plotly",
    session         = session,
    selected_idx    = selected_idx,
    sel_mode_input  = reactive(input$sel_mode),
    clear_btn_input = reactive(input$clear_sel)
  )
  output$sel_count <- renderText(paste0("已选 spots: ", length(selected_idx())))

  distance_df <- reactive({
    req(rv$pos)
    compute_signed_distance_to_selection(rv$pos, selected_idx())
  })


  setup_download_selected(
    output         = output,
    id             = "dl_sel",
    pos_r          = reactive(rv$pos),
    expr_vec_r     = expr_vec,
    selected_idx_r = reactive(selected_idx())
  )

  output$dl_distance <- downloadHandler(
    filename = function() paste0("spots_with_signed_distance_", format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      df <- distance_df()
      # 如果需要把 expr 一并导出：
      df$expr <- expr_vec()
      utils::write.csv(df[, c("barcode","x_hires","y_hires","category1","distance1","expr")],
                      file, row.names = FALSE)
    }
  )



  # ---- 调试信息（可留可去） ----
  output$dbg <- renderPrint({
    list(
      have_counts = !is.null(rv$counts),
      have_pos    = !is.null(rv$pos),
      have_scale  = !is.null(rv$hires_scale),
      have_img    = !is.null(rv$img),
      img_class   = if (!is.null(rv$img)) class(rv$img)
    )
  })
}


In [6]:
shinyApp(ui, server)


Listening on http://127.0.0.1:6709

Warning message:
“The 'plotly_selected' event tied a source ID of '' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Loading counts ...

Loading positions ...

Loading scalefactor ...

Loading image ...

